In [1]:
import os
import torch
if(not os.getcwd().endswith("EpiLLM")):
    os.chdir("..")
print(os.getcwd())
env = ! cat .env
try : 
    DEVICE = [_2.replace("\"", "") for (_1, _2) in [e.split("=") for e in env] if _1 == "DEVICE"][0]
    DEVICE = torch.device(DEVICE)
except:
    print("Set the device in .env file")
print("Device : ", DEVICE)
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

/home/antoinemagron/smb_share/TeamMembers/Magron_Antoine/EpiLLM
Device :  cuda


In [2]:
from gtts import gTTS
from pathlib import Path
import IPython.display as ipd

AUDIO_BASE_PATH = Path("./docs/audios")

def to_audio_file(gentext: str,
                  fname: str,
                  lang: str) -> str:
    audio = gTTS(text=gentext, lang=lang, slow=False)
    audio.save(AUDIO_BASE_PATH / fname)
    return AUDIO_BASE_PATH / fname

def read_audio_file(path: str) -> None:
    display(ipd.Audio(filename=path))


read_audio_file(to_audio_file("This is tentative test", "test.mp3", "en"))

In [3]:
from models import (HF_LLM, GenerationArg)
from models.qa_prompts import QA_PROMPTS
from evaluation import MCQBenchmark



def cot1_prompt_template(n_shots: int, q: str) -> str:
    template = QA_PROMPTS["1cot_answer_align"]
    shots = '\n'.join(template['shots'][:n_shots])
    return f"""-system:\n{template['system']}\n{shots}{template['q_form'].format(q=q)}"""


mcq_test = MCQBenchmark(
    "docs/benchmarks/self_assessment/aes7_processed.json",
    prompt_template=lambda : ()
).mcq[0]

prompt = cot1_prompt_template(3, mcq_test)

print(mcq_test["question"])
print(mcq_test["answer"])

We've set API key :  f0555591d0410671711554a53411c6d7
Based on the ILAE consensus classification, which of the following findings are seen in the most common histopathological subtype of hippocampal sclerosis?
A. Neuronal cell loss in the CA2 and CA3 regions
B. Neuronal cell loss in the CA1 and CA3 regions
C. Neuronal cell loss in the CA1 and CA4 regions
D. Neuronal cell loss in the dentate regions
C


In [4]:
args = GenerationArg(
    temperature=0.001,
    topk=1,
    topp=1,
    max_new_token=512,
    stop_seq="<|STOP|>",
    use_vllm=True
)
meditron = HF_LLM(
    model_name="epfl-llm/meditron-7b",
    device=DEVICE,
    use_vllm=True,
    arg=args)
meditron.load()

INFO 04-30 15:16:04 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='epfl-llm/meditron-7b', tokenizer='epfl-llm/meditron-7b', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-30 15:16:05 selector.py:51] Cannot use FlashAttention because the package is not found. Please install it for better performance.
INFO 04-30 15:16:05 selector.py:25] Using XFormers backend.
INFO 04-30 15:16:06 weight_utils.py:177] Using model weights format ['*.safetensors']
INFO 04-30 15:16:07 model_runner.py:104] Loading model weights took 12.5527 GB
INFO 04-30 15:16:08 gpu_executor.py:94] # GPU blocks: 3796, # CPU blocks: 512
INFO 04-30 15:16:08 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-30 15:16:08 model_runner.py:795] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-30 15:16:14 model_runner.py:867] G

In [10]:
print(question := mcq_test["question"])
read_audio_file(
    to_audio_file(
        question,
        "mcq0_question.mp3", "en"
    )
)

Based on the ILAE consensus classification, which of the following findings are seen in the most common histopathological subtype of hippocampal sclerosis?
A. Neuronal cell loss in the CA2 and CA3 regions
B. Neuronal cell loss in the CA1 and CA3 regions
C. Neuronal cell loss in the CA1 and CA4 regions
D. Neuronal cell loss in the dentate regions


In [9]:
print(answer := meditron.query([prompt])[0].outputs[0].text)
read_audio_file(
    to_audio_file(
    answer,
    "mcq0_answer.mp3",
    "en"
    )
)


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.22s/it]


 The most common type of pyramidal cell loss in patients with temporal lobe epilepsy is Type 1, affecting both the CA4 and CA1 sectors. This type is more often associated with a history of initial precipitating injuries before 5 years of age, with early seizure onset and favorable postsurgical seizure control. Damage to sectors CA3 and CA2 is more variable but frequently visible. Type 2 involves predominantly CA1, while Type 3 involves predominantly CA4.
Therefore, the correct answer is C.
